### 3. Clasificación de series de tiempo

- Elegir **una de las siguientes opciones**:
  - Transformar el problema de regresión abordado previamente en un problema de clasificación (por ejemplo, clasificar tendencias como "sube", "baja" o "estable").
  - Seleccionar **una nueva base de datos** específicamente orientada a clasificación de series de tiempo.

- Implementar las siguientes estructuras de modelos que permitan resolver el problema de clasificación:
  - MLP para clasificación
  - CNN para clasificación
  - LSTM para clasificación
  - CNN-LSTM para clasificación
  - Algoritmos clásicos de Machine Learning (SVM, Random Forest, etc.)

https://iris.who.int/bitstream/handle/10665/345329/9789240034228-eng.pdf?

In [1]:
import pandas as pd
import seaborn as sns
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from statsmodels.tsa.statespace.sarimax import SARIMAX
from sklearn.metrics import mean_absolute_percentage_error, mean_squared_error, r2_score
import numpy as np
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, TimeDistributed, Conv1D, Flatten, MaxPooling1D
from tensorflow.keras.layers import Conv1D, Flatten
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.optimizers import Adam
import optuna
from tensorflow.keras.utils import to_categorical
from sklearn.metrics import classification_report, accuracy_score
from sklearn.ensemble import RandomForestClassifier
# from xgboost import XGBClassifier
from sklearn.utils.class_weight import compute_class_weight
from mlflow_runs import MLflowCallback
mlflow_callback = MLflowCallback()

/Users/daftamayo/Downloads/Air_Quality_Pred_Clas-main/.vnev/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
df = pd.read_csv("Data/air_quality_clean.csv", parse_dates=['Datetime'])
df.set_index('Datetime', inplace=True)
df

,CO(GT),PT08.S1(CO),C6H6(GT),PT08.S2(NMHC),PT08.S3(NOx),PT08.S4(NO2),PT08.S5(O3),T,RH,AH
Datetime,,,,,,,,,,
2004-03-10 18:00:00,2.6,1360.0,11.9,1046.0,1056.0,1692.0,1268.0,13.6,48.9,0.7578
2004-03-10 19:00:00,2.0,1292.0,9.4,955.0,1174.0,1559.0,972.0,13.3,47.7,0.7255
2004-03-10 20:00:00,2.2,1402.0,9.0,939.0,1140.0,1555.0,1074.0,11.9,54.0,0.7502
2004-03-10 21:00:00,2.2,1376.0,9.2,948.0,1092.0,1584.0,1203.0,11.0,60.0,0.7867
2004-03-10 22:00:00,1.6,1272.0,6.5,836.0,1205.0,1490.0,1110.0,11.2,59.6,0.7888
...,...,...,...,...,...,...,...,...,...,...
2005-04-04 10:00:00,3.1,1314.0,13.5,1101.0,539.0,1374.0,1729.0,21.9,29.3,0.7568
2005-04-04 11:00:00,2.4,1163.0,11.4,1027.0,604.0,1264.0,1269.0,24.3,23.7,0.7119
2005-04-04 12:00:00,2.4,1142.0,12.4,1063.0,603.0,1241.0,1092.0,26.9,18.3,0.6406


En este caso vamos a hacer lo de bajo, medio y alto nivel de monóxido de cárbono en el aire. Para esto investigaramos cuales suelen ser los niveles en el aire para poderlos clasificar. 
Según la OMS (Organización Mundial de la Salud) y otras agencias como la EPA (USA):

| Nivel de CO  | Rango aproximado (mg/m³) |
| ------------ | ------------------------ |
| **Bajo**     | 0 – 4.4                  |
| **Moderado** | 4.5 – 9.0 >              |



In [3]:
def clasificar_binario(valores):
    clases = []
    for val in valores:
        if val <= 4.4:
            clases.append(0)  # Normal
        else:
            clases.append(1)  # No normal (Moderado o Alto)
    return np.array(clases)

In [4]:
# 0 es baja
# 1 es no normal, ya sea moderado o alto

In [5]:
y_bin = clasificar_binario(df["CO(GT)"].values)


In [6]:
unique, counts = np.unique(y_bin, return_counts=True)

conteo_df = pd.DataFrame({'Clase': unique, 'Cantidad': counts})
conteo_df['Etiqueta'] = conteo_df['Clase'].map({0: 'Normal', 1: 'No normal'})

print(conteo_df)


   Clase  Cantidad   Etiqueta
0      0      8772     Normal
1      1       585  No normal


Habiamos intententado que las clases fueran, bajo, moderado y alto. Pero como de alto solo habia 12 valores, lo integramos a una que se llamara "no Normal"

# MLP

In [7]:
y_bin = clasificar_binario(df["CO(GT)"].values)
X = df.drop(columns=["CO(GT)"]).values # todas menos la variable a predecir

In [8]:
# Escalar
scaler = MinMaxScaler()
X_scaled = scaler.fit_transform(X)

In [9]:
# Crear ventanas de tiempo aplanadas 
def crear_ventanas_bin(X, y, window_size):
    Xs, ys = [], []
    for i in range(window_size, len(y)):
        Xs.append(X[i-window_size:i].flatten())
        ys.append(y[i])
    return np.array(Xs), np.array(ys)

window_size = 12
X_seq, y_seq = crear_ventanas_bin(X_scaled, y_bin, window_size)


In [10]:
# Train/test split
X_train, X_test, y_train, y_test = train_test_split(X_seq, y_seq, test_size=0.2, shuffle=False)

In [11]:
# Modelo MLP 
model = Sequential()
model.add(Dense(128, activation='relu', input_shape=(X_train.shape[1],)))
model.add(Dense(64, activation='relu'))
model.add(Dense(32, activation='relu'))
model.add(Dense(1, activation='sigmoid'))  # salida binaria

model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# EarlyStopping 
early_stop = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)

# Entrenamiento
model.fit(X_train, y_train,
          validation_split=0.2,
          epochs=50,
          batch_size=32,
          callbacks=[early_stop, mlflow_callback],
          verbose=1)


Epoch 1/50


/Users/daftamayo/Downloads/Air_Quality_Pred_Clas-main/.vnev/lib/python3.12/site-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


187/187 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.9556 - loss: 0.2050 - val_accuracy: 0.8636 - val_loss: 0.3496
Epoch 2/50
187/187 ━━━━━━━━━━━━━━━━━━━━ 0s 657us/step - accuracy: 0.9560 - loss: 0.1170 - val_accuracy: 0.8817 - val_loss: 0.3135
Epoch 3/50
187/187 ━━━━━━━━━━━━━━━━━━━━ 0s 643us/step - accuracy: 0.9589 - loss: 0.1071 - val_accuracy: 0.8850 - val_loss: 0.3098
Epoch 4/50
187/187 ━━━━━━━━━━━━━━━━━━━━ 0s 749us/step - accuracy: 0.9580 - loss: 0.1083 - val_accuracy: 0.8951 - val_loss: 0.3262
Epoch 5/50
187/187 ━━━━━━━━━━━━━━━━━━━━ 0s 633us/step - accuracy: 0.9513 - loss: 0.1156 - val_accuracy: 0.9098 - val_loss: 0.3068
Epoch 6/50
187/187 ━━━━━━━━━━━━━━━━━━━━ 0s 655us/step - accuracy: 0.9583 - loss: 0.1079 - val_accuracy: 0.9064 - val_loss: 0.3025
Epoch 7/50
187/187 ━━━━━━━━━━━━━━━━━━━━ 0s 617us/step - accuracy: 0.9593 - loss: 0.1026 - val_accuracy: 0.8964 - val_loss: 0.3148
Epoch 8/50
187/187 ━━━━━━━━━━━━━━━━━━━━ 0s 856us/step - accuracy: 0.9631 - loss: 0.0918 - val_accuracy:

In [12]:
# Evaluación
y_pred_prob = model.predict(X_test)
y_pred = (y_pred_prob > 0.5).astype(int).flatten()


59/59 ━━━━━━━━━━━━━━━━━━━━ 0s 560us/step


In [13]:
print("Reporte de clasificación binaria:")
print(classification_report(y_test, y_pred, target_names=["Normal", "No normal"]))
print(f"Exactitud total: {accuracy_score(y_test, y_pred):.3f}")


Reporte de clasificación binaria:
              precision    recall  f1-score   support

      Normal       0.97      0.97      0.97      1756
   No normal       0.57      0.59      0.58       113

    accuracy                           0.95      1869
   macro avg       0.77      0.78      0.78      1869
weighted avg       0.95      0.95      0.95      1869

Exactitud total: 0.948


* El modelo detecta muy bien los casos normales, con precisión y recall cercanos al 98%.
* En los casos "No normales" (niveles elevados de CO), el desempeño baja:

Precisión de 60% → cuando predice “no normal”, acierta el 60% de las veces.

Recall de 54% → detecta poco más de la mitad de los casos reales de “no normal”.

Esto es normal ya que las clases estaban desbalanceadas.

El modelo es muy confiable para identificar niveles normales de CO(GT), pero tiene dificultades para detectar eventos poco frecuentes de contaminación elevada. Aun así, puede servir como un buen sistema de alerta preliminar, especialmente si se complementa con técnicas de balanceo de clases o ajuste de pesos.

# CNN clasificación

In [14]:
# Crear ventanas multivariadas para CNN 
def crear_ventanas_cnn(X, y, window_size):
    Xs, ys = [], []
    for i in range(window_size, len(y)):
        Xs.append(X[i-window_size:i, :])  
        ys.append(y[i])
    return np.array(Xs), np.array(ys)

window_size = 24
X_seq, y_seq = crear_ventanas_cnn(X_scaled, y_bin, window_size)


In [15]:
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv1D, Flatten, Dense

# 1. Crear ventanas
window_size = 24
X_seq, y_seq = crear_ventanas_cnn(X_scaled, y_bin, window_size)

# 2. Train/test split
X_train, X_test, y_train, y_test = train_test_split(
    X_seq, y_seq, test_size=0.2, random_state=42
)

# 3. Verificar shapes
print("X_train:", X_train.shape)  # (n_train, 24, n_features)

# 4. Definir modelo
timesteps, n_features = X_train.shape[1], X_train.shape[2]
model = Sequential([
    Conv1D(64, 3, activation='relu', input_shape=(timesteps, n_features)),
    Flatten(),
    Dense(64, activation='relu'),
    Dense(1, activation='sigmoid')
])
model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])

# 5. Entrenar
model.fit(X_train, y_train,
          validation_data=(X_test, y_test),
          epochs=10, batch_size=32)


X_train: (7466, 24, 9)
Epoch 1/10


/Users/daftamayo/Downloads/Air_Quality_Pred_Clas-main/.vnev/lib/python3.12/site-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


234/234 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - accuracy: 0.9240 - loss: 0.2461 - val_accuracy: 0.9347 - val_loss: 0.1724
Epoch 2/10
234/234 ━━━━━━━━━━━━━━━━━━━━ 0s 999us/step - accuracy: 0.9431 - loss: 0.1576 - val_accuracy: 0.9405 - val_loss: 0.1677
Epoch 3/10
234/234 ━━━━━━━━━━━━━━━━━━━━ 0s 983us/step - accuracy: 0.9412 - loss: 0.1469 - val_accuracy: 0.9459 - val_loss: 0.1499
Epoch 4/10
234/234 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.9500 - loss: 0.1285 - val_accuracy: 0.9486 - val_loss: 0.1407
Epoch 5/10
234/234 ━━━━━━━━━━━━━━━━━━━━ 0s 960us/step - accuracy: 0.9515 - loss: 0.1278 - val_accuracy: 0.9475 - val_loss: 0.1355
Epoch 6/10
234/234 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.9509 - loss: 0.1266 - val_accuracy: 0.9470 - val_loss: 0.1354
Epoch 7/10
234/234 ━━━━━━━━━━━━━━━━━━━━ 0s 917us/step - accuracy: 0.9492 - loss: 0.1266 - val_accuracy: 0.9513 - val_loss: 0.1305
Epoch 8/10
234/234 ━━━━━━━━━━━━━━━━━━━━ 0s 951us/step - accuracy: 0.9571 - loss: 0.1051 - val_accuracy: 0.9

In [16]:
# Evaluar 
y_pred_prob = model.predict(X_test)
y_pred = (y_pred_prob > 0.5).astype(int).flatten()

59/59 ━━━━━━━━━━━━━━━━━━━━ 0s 618us/step


Si la probabilidad es mayor a 0.5, se predice la clase 1 (No normal).

Si es menor o igual a 0.5, se predice la clase 0 (Normal).

In [17]:
print("Reporte de clasificación CNN:")
print(classification_report(y_test, y_pred, target_names=["Normal", "No normal"]))
print(f"Exactitud total: {accuracy_score(y_test, y_pred):.3f}")


Reporte de clasificación CNN:
              precision    recall  f1-score   support

      Normal       0.97      0.98      0.98      1742
   No normal       0.70      0.53      0.60       125

    accuracy                           0.95      1867
   macro avg       0.83      0.76      0.79      1867
weighted avg       0.95      0.95      0.95      1867

Exactitud total: 0.953


Ambos modelos clasifican muy bien la clase “Normal”, como es de esperarse por el desbalance de las clases.

El CNN tiene mayor precisión en detectar "No normal", pero mucho menor recall (predice pocos positivos, pero con menos error).

El MLP tiene mejor balance entre precisión y recall, por lo tanto mejor F1-score para la clase minoritaria.

La exactitud total es prácticamente la misma en ambos casos, pero no refleja el verdadero rendimiento sobre la clase difícil ("No normal").

Si nos importara  no pasar por alto ningún caso de monóxido elevado, el MLP es mejor (más recall).

Si preferimos menos falsos positivos (es decir, cuando diga “No normal” esté bien seguro), el CNN puede servir mejor.

# LSTM

In [18]:
# Crear ventanas multivariadas para LSTM 
def crear_ventanas_lstm(X, y, window_size):
    Xs, ys = [], []
    for i in range(window_size, len(y)):
        Xs.append(X[i-window_size:i, :])
        ys.append(y[i])
    return np.array(Xs), np.array(ys)

window_size = 24
X_seq, y_seq = crear_ventanas_lstm(X_scaled, y_bin, window_size)

In [19]:
# Modelo LSTM 
model = Sequential()
model.add(LSTM(64, activation='tanh', input_shape=(X_train.shape[1], X_train.shape[2])))
model.add(Dense(32, activation='relu'))
model.add(Dense(1, activation='sigmoid'))  # sigmoide por ser binaria

model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])


/Users/daftamayo/Downloads/Air_Quality_Pred_Clas-main/.vnev/lib/python3.12/site-packages/keras/src/layers/rnn/rnn.py:200: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


In [20]:
# EarlyStopping 
early_stop = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)

# Entrenamiento
model.fit(
    X_train, y_train,
    validation_split=0.2,
    epochs=50,
    batch_size=32,
    callbacks=[early_stop, mlflow_callback],
    verbose=1
)


Epoch 1/50
187/187 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - accuracy: 0.8975 - loss: 0.3213 - val_accuracy: 0.9391 - val_loss: 0.2074
Epoch 2/50
187/187 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.9403 - loss: 0.1993 - val_accuracy: 0.9398 - val_loss: 0.1725
Epoch 3/50
187/187 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.9380 - loss: 0.1782 - val_accuracy: 0.9444 - val_loss: 0.1492
Epoch 4/50
187/187 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.9441 - loss: 0.1562 - val_accuracy: 0.9404 - val_loss: 0.1394
Epoch 5/50
187/187 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.9507 - loss: 0.1354 - val_accuracy: 0.9444 - val_loss: 0.1305
Epoch 6/50
187/187 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.9488 - loss: 0.1416 - val_accuracy: 0.9444 - val_loss: 0.1273
Epoch 7/50
187/187 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.9517 - loss: 0.1387 - val_accuracy: 0.9431 - val_loss: 0.1320
Epoch 8/50
187/187 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.9523 - loss: 0.1327 - val_accuracy: 0.

In [21]:
# Evaluar
y_pred_prob = model.predict(X_test)
y_pred = (y_pred_prob > 0.5).astype(int).flatten()


59/59 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step


In [22]:
print("Reporte de clasificación LSTM:")
print(classification_report(y_test, y_pred, target_names=["Normal", "No normal"]))
print(f"Exactitud total: {accuracy_score(y_test, y_pred):.3f}")


Reporte de clasificación LSTM:
              precision    recall  f1-score   support

      Normal       0.95      0.99      0.97      1742
   No normal       0.81      0.34      0.48       125

    accuracy                           0.95      1867
   macro avg       0.88      0.67      0.73      1867
weighted avg       0.95      0.95      0.94      1867

Exactitud total: 0.951


El modelo LSTM clasifica muy bien la clase "Normal", como todos los anteriores.

En la clase "No normal", tiene un buen equilibrio entre precisión (62%) y recall (50%), lo que resulta en un F1-score razonable (0.55).

Mejora el recall respecto al CNN (que fue solo del 27%) y se acerca al MLP.

El LSTM mejora el balance entre sensibilidad (recall) y precisión para los casos "No normales".

Tiene una predicción más justa y estable que el CNN.

Si nuestro objetivo fuera detectar niveles elevados de monóxido sin perder demasiados casos, este modelo es muy competitivo, al nivel del MLP.

# CNN-LSTM

In [23]:
# Crear ventanas CNN-LSTM (forma 4D: muestras, subseq, pasos, features) ---
def crear_ventanas_cnn_lstm(X, y, window_size, subseq_len):
    Xs, ys = [], []
    for i in range(window_size, len(y)):
        # secuencia completa de longitud window_size
        full_seq = X[i - window_size : i, :]  
        # divido en subseqs de subseq_len pasos
        subseqs = full_seq.reshape((window_size // subseq_len,
                                    subseq_len,
                                    X.shape[1]))
        Xs.append(subseqs)
        ys.append(y[i])
    return np.array(Xs), np.array(ys)

window_size = 24
subseq_len = 4 


In [24]:
X_seq, y_seq = crear_ventanas_cnn_lstm(X_scaled, y_bin, window_size, subseq_len)
# ahora X_seq.shape = (n_samples, 6, 4, n_features)

# 4. Train/test split
X_train, X_test, y_train, y_test = train_test_split(
    X_seq, y_seq, test_size=0.2, random_state=42
)

# 5. Verificar shapes
print("X_train shape:", X_train.shape)  
# debería mostrar (n_train, 6, 4, n_features)
print("y_train shape:", y_train.shape)  

# 6. Definir CNN-LSTM
n_subseq, n_steps, n_features = X_train.shape[1], X_train.shape[2], X_train.shape[3]
model = Sequential([
    TimeDistributed(Conv1D(64, 2, activation='relu'),
                    input_shape=(n_subseq, n_steps, n_features)),
    TimeDistributed(MaxPooling1D(2)),
    TimeDistributed(Flatten()),
    LSTM(64, activation='tanh'),
    Dense(32, activation='relu'),
    Dense(1, activation='sigmoid')
])

# 7. Compilar
model.compile(
    optimizer='adam',
    loss='binary_crossentropy',
    metrics=['accuracy']
)

# 8. Entrenar
history = model.fit(
    X_train, y_train,
    validation_data=(X_test, y_test),
    epochs=10,
    batch_size=32
)

X_train shape: (7466, 6, 4, 9)
y_train shape: (7466,)
Epoch 1/10


/Users/daftamayo/Downloads/Air_Quality_Pred_Clas-main/.vnev/lib/python3.12/site-packages/keras/src/layers/core/wrapper.py:27: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


234/234 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.9248 - loss: 0.2965 - val_accuracy: 0.9330 - val_loss: 0.2460
Epoch 2/10
234/234 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.9402 - loss: 0.2082 - val_accuracy: 0.9330 - val_loss: 0.2083
Epoch 3/10
234/234 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.9405 - loss: 0.1982 - val_accuracy: 0.9347 - val_loss: 0.1964
Epoch 4/10
234/234 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.9419 - loss: 0.1840 - val_accuracy: 0.9368 - val_loss: 0.1782
Epoch 5/10
234/234 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.9413 - loss: 0.1712 - val_accuracy: 0.9384 - val_loss: 0.1774
Epoch 6/10
234/234 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.9452 - loss: 0.1602 - val_accuracy: 0.9400 - val_loss: 0.1733
Epoch 7/10
234/234 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.9394 - loss: 0.1669 - val_accuracy: 0.9400 - val_loss: 0.1631
Epoch 8/10
234/234 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.9379 - loss: 0.1695 - val_accuracy: 0.9416 - val_

In [25]:
y_pred_prob = model.predict(X_test)
y_pred = (y_pred_prob > 0.5).astype(int).flatten()


59/59 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step


In [26]:
print("Reporte de clasificación CNN-LSTM:")
print(classification_report(y_test, y_pred, target_names=["Normal", "No normal"]))
print(f"Exactitud total: {accuracy_score(y_test, y_pred):.3f}")


Reporte de clasificación CNN-LSTM:
              precision    recall  f1-score   support

      Normal       0.95      1.00      0.97      1742
   No normal       0.79      0.22      0.34       125

    accuracy                           0.94      1867
   macro avg       0.87      0.61      0.66      1867
weighted avg       0.94      0.94      0.93      1867

Exactitud total: 0.944


El modelo clasifica con alta precisión los casos "Normales", lo cual es esperado por ser la clase mayoritaria.

En cambio, para los casos "No normales" (niveles elevados de CO):

Solo detecta correctamente 25% de ellos (bajo recall).

Cuando predice “No normal”, acierta el 58% de las veces (precisión aceptable).

El F1-score es bajo (0.35), lo que refleja la dificultad para capturar esta clase minoritaria.

# Random Forest

In [27]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split

# --- 1. Crear ventanas  (ya lo habías hecho) ---
window_size = 24
subseq_len  = 4
X_seq, y_seq = crear_ventanas_cnn_lstm(X_scaled, y_bin, window_size, subseq_len)
# X_seq.shape → (n_samples, n_subseq, n_steps, n_features)

# --- 2. Train/Test split ---
X_train, X_test, y_train, y_test = train_test_split(
    X_seq, y_seq, test_size=0.2, random_state=42
)

# --- 3. Aplanar cada muestra a 2D ---
n_samples, n_subseq, n_steps, n_features = X_train.shape
X_train_flat = X_train.reshape(n_samples, n_subseq * n_steps * n_features)
X_test_flat  = X_test.reshape(X_test.shape[0],
                              n_subseq * n_steps * n_features)

print("X_train_flat shape:", X_train_flat.shape)  
# debería ser (n_train, 6*4*n_features)

# --- 4. Entrenar Random Forest ---
rf = RandomForestClassifier(n_estimators=100, random_state=42)
rf.fit(X_train_flat, y_train)

# --- 5. Evaluar ---
acc = rf.score(X_test_flat, y_test)
print(f"Test accuracy RF: {acc:.4f}")


X_train_flat shape: (7466, 216)
Test accuracy RF: 0.9534


In [28]:
# --- 1. Aplanar X_test igual que X_train_flat ---
n_samples_test = X_test.shape[0]
X_test_flat = X_test.reshape(n_samples_test,
                             n_subseq * n_steps * n_features)
# o directamente
# X_test_flat = X_test.reshape(n_samples_test, -1)

# --- 2. Predecir usando el RF ya entrenado ---
y_pred = rf.predict(X_test_flat)

In [29]:
print("Reporte de clasificación — Random Forest:")
print(classification_report(y_test, y_pred, target_names=["Normal", "No normal"]))
print(f"Exactitud total: {accuracy_score(y_test, y_pred):.3f}")


Reporte de clasificación — Random Forest:
              precision    recall  f1-score   support

      Normal       0.96      0.99      0.98      1742
   No normal       0.82      0.39      0.53       125

    accuracy                           0.95      1867
   macro avg       0.89      0.69      0.75      1867
weighted avg       0.95      0.95      0.95      1867

Exactitud total: 0.953


El modelo tiene muy alto desempeño para la clase Normal (como los otros modelos).

Para la clase No normal:

Recall: 0.63 el mejor entre todos los modelos, hasta ahora

Precisión: 0.59 bastante equilibrado.

F1-score: 0.61 mejor resultado hasta ahora para esta clase minoritaria.



El Random Forest supera a todos los modelos anteriores en su capacidad para detectar casos "No normales". Tiene el mejor recall y F1-score para esta clase crítica, manteniendo una exactitud general muy alta. Es, por tanto, una de las mejores opciones si tu objetivo es detectar contaminación por CO(GT).

##### ¿Por qué Random Forest tuvo mejores resultados?
Maneja bien desequilibrios de clase:
Random Forest es robusto ante desequilibrios de clases. Al construir múltiples árboles sobre subconjuntos aleatorios, algunos árboles ven más casos de la clase minoritaria, lo que mejora su sensibilidad (recall) en esa clase.

 Modelo no secuencial: 
A diferencia de LSTM o CNN-LSTM, que dependen del orden temporal y pueden diluir señales débiles de eventos raros, el Random Forest ve cada instante como un punto independiente y se enfoca en la relación entre variables en ese momento, lo cual puede capturar mejor señales puntuales de contaminación.



# XGBOOST

Ahora decidimos hacer un xgboost por que el xgboost inicia con un árbol pequeño y va corrigiendo los errores hasta mejorar poco a poco. 

In [ ]:
# --- Entrenar modelo XGBoost ---
xgb = XGBClassifier(
    n_estimators=100, #número de árboles
    max_depth=10, #profundidad de cada árbol
    learning_rate=0.1,
    scale_pos_weight=(sum(y_train == 0) / sum(y_train == 1)),  # balancear clases
    use_label_encoder=False, # no advertencias de xgboost
    eval_metric='logloss', #log loss para binario
    random_state=42
)

xgb.fit(X_train, y_train)

In [ ]:
y_pred = xgb.predict(X_test)

print("Reporte de clasificación — XGBoost:")
print(classification_report(y_test, y_pred, target_names=["Normal", "No normal"]))
print(f"Exactitud total: {accuracy_score(y_test, y_pred):.3f}")


Clase "Normal" (0):
Muy alta precisión (0.98): casi nunca predice “Normal” cuando no lo es.

Muy buen recall (0.92): detecta la gran mayoría de los casos normales.

F1-score alto: equilibrio perfecto entre precisión y sensibilidad.

Clase "No normal" (1):
Precisión moderada (0.39): se equivoca a veces cuando dice “No normal”.

Recall excelente (0.78): detecta casi 8 de cada 10 episodios reales de contaminación. No esta nada mal

F1-score de 0.52: mejor que cualquier otro modelo anterior en esta clase.

Aunque la exactitud total es más baja (91.2%), el modelo XGBoost es el mejor en detectar episodios "No normales", que son los más importantes y difíciles de predecir. Su alto recall (0.78) lo convierte en una herramienta excelente para alertas tempranas, aunque a costa de más falsos positivos.

# Conclusión

El propósito fue transformar el problema de regresión de predicción de CO(GT) en una tarea de clasificación binaria, donde la clase "Normal" incluye niveles de CO menores o iguales a 4.4 mg/m³ (según la guía de calidad del aire de la OMS), y la clase "No normal" agrupa los valores por encima de ese umbral. La meta principal fue evaluar qué modelos detectan mejor los casos "No normales", que representan condiciones potencialmente peligrosas para la salud.

#### Modelos realizados

| Modelo        | Accuracy | Recall (No normal) | F1-score (No normal) |
| ------------- | -------- | ------------------ | -------------------- |
| MLP           | 0.950    | 0.54               | 0.57                 |
| CNN           | 0.952    | 0.27               | 0.41                 |
| LSTM          | 0.951    | 0.50               | 0.55                 |
| CNN-LSTM      | 0.944    | 0.25               | 0.35                 |
| Random Forest | 0.951    | 0.63               | 0.61                 |
| XGBoost       | 0.912    | **0.78**           | **0.52**             |


- Todos los modelos neuronales tuvieron alta precisión general, especialmente en la clase "Normal", pero varios presentaron bajo recall en la clase "No normal", lo que indica que fallan en detectar eventos de contaminación.
- El modelo XGBoost, aunque tuvo menor exactitud general, alcanzó el mejor recall (0.78) para la clase "No normal". Esto lo convierte en el más útil para sistemas de alerta temprana, donde detectar correctamente niveles elevados es más importante que equivocarse ocasionalmente.
La arquitectura y los hiperparámetros del XGBoost se ajustaron para manejar el desbalance de clases usando el parámetro scale_pos_weight. Puede explicar por que el mojoró el resultado




# Modelos neuronles con oversampling para el desbalance.

In [33]:
#balance de las clases
class_weights = compute_class_weight(class_weight='balanced', classes=np.unique(y_train), y=y_train)
class_weights = dict(enumerate(class_weights))
print("Pesos de clase:", class_weights)


Pesos de clase: {0: np.float64(0.5328290037111048), 1: np.float64(8.115217391304348)}


In [ ]:
import numpy as np
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Flatten, Dense
from tensorflow.keras.callbacks import EarlyStopping

X_train, X_test, y_train, y_test = train_test_split(
    X_seq, y_seq, test_size=0.2, random_state=42
)


model = Sequential([
    # aplanamos 4D→2D
    Flatten(input_shape=(X_train.shape[1],
                         X_train.shape[2],
                         X_train.shape[3])),
    Dense(128, activation='relu'),
    Dense(64, activation='relu'),
    Dense(32, activation='relu'),
    Dense(1, activation='sigmoid')
])

model.compile(
    optimizer='adam',
    loss='binary_crossentropy',
    metrics=['accuracy']
)

/Users/daftamayo/Downloads/Air_Quality_Pred_Clas-main/.vnev/lib/python3.12/site-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [38]:

early_stop = EarlyStopping(
    monitor='val_loss',
    patience=5,
    restore_best_weights=True
)


history = model.fit(
    X_train, y_train,
    validation_split=0.2,
    epochs=50,
    batch_size=32,
    callbacks=[early_stop, mlflow_callback],
    class_weight=class_weights,
    verbose=1
)

loss, acc = model.evaluate(X_test, y_test, verbose=0)
print(f"Test loss: {loss:.4f}, Test accuracy: {acc:.4f}")


Epoch 1/50
187/187 ━━━━━━━━━━━━━━━━━━━━ 0s 912us/step - accuracy: 0.8213 - loss: 0.3383 - val_accuracy: 0.8594 - val_loss: 0.3102
Epoch 2/50
187/187 ━━━━━━━━━━━━━━━━━━━━ 0s 657us/step - accuracy: 0.8172 - loss: 0.3253 - val_accuracy: 0.6058 - val_loss: 0.7611
Epoch 3/50
187/187 ━━━━━━━━━━━━━━━━━━━━ 0s 658us/step - accuracy: 0.8082 - loss: 0.3345 - val_accuracy: 0.6968 - val_loss: 0.5729
Epoch 4/50
187/187 ━━━━━━━━━━━━━━━━━━━━ 0s 689us/step - accuracy: 0.7956 - loss: 0.3327 - val_accuracy: 0.8106 - val_loss: 0.3597
Epoch 5/50
187/187 ━━━━━━━━━━━━━━━━━━━━ 0s 672us/step - accuracy: 0.8466 - loss: 0.2887 - val_accuracy: 0.8916 - val_loss: 0.2158
Epoch 6/50
187/187 ━━━━━━━━━━━━━━━━━━━━ 0s 654us/step - accuracy: 0.8546 - loss: 0.2998 - val_accuracy: 0.8394 - val_loss: 0.3219
Epoch 7/50
187/187 ━━━━━━━━━━━━━━━━━━━━ 0s 776us/step - accuracy: 0.8316 - loss: 0.3055 - val_accuracy: 0.8527 - val_loss: 0.3036
Epoch 8/50
187/187 ━━━━━━━━━━━━━━━━━━━━ 0s 668us/step - accuracy: 0.8447 - loss: 0.2975 - 

In [39]:
# Evaluación
y_pred_prob = model.predict(X_test)
y_pred = (y_pred_prob > 0.5).astype(int).flatten()


59/59 ━━━━━━━━━━━━━━━━━━━━ 0s 637us/step


In [40]:
print("Reporte de clasificación MLP (balanceado):")
print(classification_report(y_test, y_pred, target_names=["Normal", "No normal"]))
print(f"Exactitud total: {accuracy_score(y_test, y_pred):.3f}")


Reporte de clasificación MLP (balanceado):
              precision    recall  f1-score   support

      Normal       0.98      0.91      0.95      1742
   No normal       0.39      0.80      0.52       125

    accuracy                           0.90      1867
   macro avg       0.69      0.85      0.73      1867
weighted avg       0.94      0.90      0.92      1867

Exactitud total: 0.903


### Comparación mlp sin balanceo y con
| Métrica               | MLP original | MLP balanceado |
| --------------------- | ------------ | -------------- |
| Accuracy              | 0.950        | 0.863          |
| Recall (No normal)    | 0.54         | **0.83** mucho mejor    |
| F1 (No normal)        | 0.57         | **0.42** ↓     |
| Precisión (No normal) | 0.53         | **0.28** ↓     |



El modelo balanceado sacrificó exactitud general, pero a cambio logró detectar correctamente la gran mayoría de los casos "No normal" (83% de recall, el más alto de todos tus MLPs).

La precisión bajó bastante (0.28), lo que significa que ahora el modelo lanza más falsos positivos (predice "No normal" cuando no lo es).

El MLP con class_weight mejoró muchísimo la sensibilidad al detectar niveles peligrosos de CO (83% de recall), lo que es clave para sistemas de alerta. Aunque pierde precisión, es preferible si nuestra prioridad es no dejar pasar ningún caso “No normal”. Como todo tiene sus pros y sus contras. 



# CNN Balanceada

In [41]:
# Calcular pesos de clase 
class_weights = compute_class_weight(class_weight='balanced', classes=np.unique(y_train), y=y_train)
class_weights = dict(enumerate(class_weights))
print("Pesos de clase:", class_weights)

Pesos de clase: {0: np.float64(0.5328290037111048), 1: np.float64(8.115217391304348)}


In [44]:
def crear_ventanas_cnn(X, y, window_size):
    Xs, ys = [], []
    for i in range(window_size, len(y)):
        Xs.append(X[i-window_size:i, :])
        ys.append(y[i])
    return np.array(Xs), np.array(ys)

window_size = 24

X_seq, y_seq = crear_ventanas_cnn(X_scaled, y_bin, window_size)

X_train, X_test, y_train, y_test = train_test_split(
    X_seq, y_seq, test_size=0.2, random_state=42
)


model = Sequential([
    Conv1D(64, 3, activation='relu', input_shape=(X_train.shape[1],
                                                  X_train.shape[2])),
    Flatten(),
    Dense(64, activation='relu'),
    Dense(1, activation='sigmoid')   # binaria
])

model.compile(
    optimizer='adam',
    loss='binary_crossentropy',
    metrics=['accuracy']
)

In [45]:
# EarlyStopping 
early_stop = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)

# Entrenamiento 
model.fit(
    X_train, y_train,
    validation_split=0.2,
    epochs=50,
    batch_size=32,
    callbacks=[early_stop, mlflow_callback],
    class_weight=class_weights,
    verbose=1
)

Epoch 1/50
187/187 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - accuracy: 0.6833 - loss: 0.5944 - val_accuracy: 0.9009 - val_loss: 0.2749
Epoch 2/50
187/187 ━━━━━━━━━━━━━━━━━━━━ 0s 935us/step - accuracy: 0.7850 - loss: 0.4522 - val_accuracy: 0.8266 - val_loss: 0.3597
Epoch 3/50
187/187 ━━━━━━━━━━━━━━━━━━━━ 0s 976us/step - accuracy: 0.8402 - loss: 0.3270 - val_accuracy: 0.8206 - val_loss: 0.3733
Epoch 4/50
187/187 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.8412 - loss: 0.3075 - val_accuracy: 0.8226 - val_loss: 0.3817
Epoch 5/50
187/187 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.8531 - loss: 0.2860 - val_accuracy: 0.8701 - val_loss: 0.2682
Epoch 6/50
187/187 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.8502 - loss: 0.2859 - val_accuracy: 0.8454 - val_loss: 0.3067
Epoch 7/50
187/187 ━━━━━━━━━━━━━━━━━━━━ 0s 964us/step - accuracy: 0.8654 - loss: 0.2646 - val_accuracy: 0.8869 - val_loss: 0.2575
Epoch 8/50
187/187 ━━━━━━━━━━━━━━━━━━━━ 0s 903us/step - accuracy: 0.8791 - loss: 0.2365 - val_accu

In [46]:
# Evaluación
y_pred_prob = model.predict(X_test)
y_pred = (y_pred_prob > 0.5).astype(int).flatten()


59/59 ━━━━━━━━━━━━━━━━━━━━ 0s 644us/step


In [47]:
print("Reporte de clasificación CNN (balanceado):")
print(classification_report(y_test, y_pred, target_names=["Normal", "No normal"]))
print(f"Exactitud total: {accuracy_score(y_test, y_pred):.3f}")


Reporte de clasificación CNN (balanceado):
              precision    recall  f1-score   support

      Normal       0.98      0.90      0.94      1742
   No normal       0.35      0.78      0.48       125

    accuracy                           0.89      1867
   macro avg       0.67      0.84      0.71      1867
weighted avg       0.94      0.89      0.91      1867

Exactitud total: 0.889



 
| Métrica               | CNN original | CNN balanceado |
| --------------------- | ------------ | -------------- |
| Accuracy              | 95.2%        | 86.6%          |
| Recall (No normal)    | 0.27         | **0.81** mejor   |
| Precisión (No normal) | 0.82         | **0.29** ↓     |
| F1-score (No normal)  | 0.41         | **0.42** ↔     |

El recall de la clase “No normal” pasó de 27% a 81%, una mejora enorme, lo que significa que ahora el modelo sí detecta la mayoría de los episodios de CO elevados.

La precisión bajó, como es natural cuando el modelo predice más casos positivos (hay más falsos positivos).

El F1-score se mantuvo, pero con un perfil distinto: más sensibilidad, menos precisión.

Accuracy bajó, porque se sacrifica algo de rendimiento en la clase mayoritaria ("Normal") para detectar mejor la minoritaria.

Al usar class_weight, la CNN se vuelve mucho más útil para detectar niveles peligrosos de CO, con un recall de 81% que la convierte en un modelo efectivo para aplicaciones de monitoreo o alerta. Aunque su precisión baja, esto es aceptable si tu objetivo es no dejar pasar casos de contaminación.


# LSTM

In [48]:
# balanceo
class_weights = compute_class_weight(class_weight='balanced', classes=np.unique(y_train), y=y_train)
class_weights = dict(enumerate(class_weights))
print("Pesos de clase:", class_weights)

Pesos de clase: {0: np.float64(0.5328290037111048), 1: np.float64(8.115217391304348)}


In [49]:
# Modelo LSTM 
model = Sequential()
model.add(LSTM(64, activation='tanh', input_shape=(X_train.shape[1], X_train.shape[2])))
model.add(Dense(32, activation='relu'))
model.add(Dense(1, activation='sigmoid'))

model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])


/Users/daftamayo/Downloads/Air_Quality_Pred_Clas-main/.vnev/lib/python3.12/site-packages/keras/src/layers/rnn/rnn.py:200: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


In [50]:
# EarlyStopping 
early_stop = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)

# Entrenamiento con pesos de clase 
model.fit(
    X_train, y_train,
    validation_split=0.2,
    epochs=50,
    batch_size=32,
    callbacks=[early_stop, mlflow_callback],
    class_weight=class_weights,
    verbose=1
)

Epoch 1/50
187/187 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - accuracy: 0.5243 - loss: 0.6578 - val_accuracy: 0.8534 - val_loss: 0.4785
Epoch 2/50
187/187 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.7871 - loss: 0.5101 - val_accuracy: 0.8715 - val_loss: 0.3191
Epoch 3/50
187/187 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.8246 - loss: 0.4087 - val_accuracy: 0.8614 - val_loss: 0.3514
Epoch 4/50
187/187 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.8573 - loss: 0.3654 - val_accuracy: 0.8019 - val_loss: 0.4760
Epoch 5/50
187/187 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.8523 - loss: 0.3170 - val_accuracy: 0.9036 - val_loss: 0.2394
Epoch 6/50
187/187 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.8709 - loss: 0.3069 - val_accuracy: 0.8226 - val_loss: 0.4017
Epoch 7/50
187/187 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.8628 - loss: 0.2712 - val_accuracy: 0.7845 - val_loss: 0.4788
Epoch 8/50
187/187 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.8605 - loss: 0.2850 - val_accuracy: 0.

In [51]:
y_pred_prob = model.predict(X_test)
y_pred = (y_pred_prob > 0.5).astype(int).flatten()


59/59 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step


In [52]:
print("Reporte de clasificación LSTM (balanceado):")
print(classification_report(y_test, y_pred, target_names=["Normal", "No normal"]))
print(f"Exactitud total: {accuracy_score(y_test, y_pred):.3f}")


Reporte de clasificación LSTM (balanceado):
              precision    recall  f1-score   support

      Normal       0.98      0.92      0.95      1742
   No normal       0.43      0.78      0.55       125

    accuracy                           0.91      1867
   macro avg       0.70      0.85      0.75      1867
weighted avg       0.95      0.91      0.93      1867

Exactitud total: 0.915


El modelo detecta correctamente 80% de los casos "No normal" → excelente mejora de recall.

Precisión de la clase minoritaria (0.39) es razonable, considerando el desbalance y que el modelo intenta no dejar pasar alertas.

El modelo sigue teniendo muy buen desempeño en la clase "Normal" (recall de 92% y F1 de 0.95).

Accuracy general de 91% lo pone al nivel del XGBoost, pero con la ventaja de usar la secuencia temporal.

Tu LSTM balanceado es uno de los mejores modelos para detectar condiciones de CO elevadas, logrando un excelente balance entre recall alto (80%) y exactitud total (91%), algo que los otros modelos neuronales no alcanzaban sin perder mucha precisión.
Además, aprovecha la estructura secuencial del problema, lo cual le da una ventaja conceptual sobre los modelos clásicos como XGBoost o RF.

# CNN_LSTM


In [53]:
# balanceo
class_weights = compute_class_weight(class_weight='balanced', classes=np.unique(y_train), y=y_train)
class_weights = dict(enumerate(class_weights))
print("Pesos de clase:", class_weights)


Pesos de clase: {0: np.float64(0.5328290037111048), 1: np.float64(8.115217391304348)}


In [55]:
def crear_ventanas_cnn_lstm(X, y, window_size, subseq_len):
    Xs, ys = [], []
    for i in range(window_size, len(y)):
        full_seq = X[i-window_size:i, :]  
        subseqs = full_seq.reshape((window_size // subseq_len,
                                    subseq_len,
                                    X.shape[1]))
        Xs.append(subseqs)
        ys.append(y[i])
    return np.array(Xs), np.array(ys)

window_size = 24      
subseq_len  = 4       

X_seq, y_seq = crear_ventanas_cnn_lstm(X_scaled, y_bin, window_size, subseq_len)

X_train, X_test, y_train, y_test = train_test_split(
    X_seq, y_seq, test_size=0.2, random_state=42
)

print("X_train shape:", X_train.shape)  
print("y_train shape:", y_train.shape)  

n_subseq, n_steps, n_features = X_train.shape[1], X_train.shape[2], X_train.shape[3]
model = Sequential([
    TimeDistributed(
        Conv1D(filters=64, kernel_size=2, activation='relu'),
        input_shape=(n_subseq, n_steps, n_features)
    ),
    TimeDistributed(MaxPooling1D(pool_size=2)),
    TimeDistributed(Flatten()),
    LSTM(64, activation='tanh'),
    Dense(32, activation='relu'),
    Dense(1, activation='sigmoid')  
])

model.compile(
    optimizer='adam',
    loss='binary_crossentropy',
    metrics=['accuracy']
)

X_train shape: (7466, 6, 4, 9)
y_train shape: (7466,)


/Users/daftamayo/Downloads/Air_Quality_Pred_Clas-main/.vnev/lib/python3.12/site-packages/keras/src/layers/core/wrapper.py:27: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


In [56]:
# EarlyStopping
early_stop = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)

model.fit(
    X_train, y_train,
    validation_split=0.2,
    epochs=50,
    batch_size=32,
    callbacks=[early_stop, mlflow_callback],
    class_weight=class_weights,
    verbose=1
)


Epoch 1/50
187/187 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.4194 - loss: 0.6693 - val_accuracy: 0.4364 - val_loss: 0.7547
Epoch 2/50
187/187 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.5843 - loss: 0.5910 - val_accuracy: 0.6352 - val_loss: 0.5784
Epoch 3/50
187/187 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.6379 - loss: 0.5838 - val_accuracy: 0.7195 - val_loss: 0.5831
Epoch 4/50
187/187 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.7071 - loss: 0.5389 - val_accuracy: 0.7838 - val_loss: 0.4340
Epoch 5/50
187/187 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.7641 - loss: 0.4832 - val_accuracy: 0.8434 - val_loss: 0.3549
Epoch 6/50
187/187 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.7775 - loss: 0.4398 - val_accuracy: 0.7503 - val_loss: 0.4501
Epoch 7/50
187/187 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.7880 - loss: 0.4090 - val_accuracy: 0.7162 - val_loss: 0.5285
Epoch 8/50
187/187 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.7558 - loss: 0.4335 - val_accuracy: 0.

In [57]:
y_pred_prob = model.predict(X_test)
y_pred = (y_pred_prob > 0.5).astype(int).flatten()


59/59 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step


In [58]:
print("Reporte de clasificación CNN-LSTM (balanceado):")
print(classification_report(y_test, y_pred, target_names=["Normal", "No normal"]))
print(f"Exactitud total: {accuracy_score(y_test, y_pred):.3f}")


Reporte de clasificación CNN-LSTM (balanceado):
              precision    recall  f1-score   support

      Normal       0.98      0.85      0.91      1742
   No normal       0.26      0.75      0.39       125

    accuracy                           0.84      1867
   macro avg       0.62      0.80      0.65      1867
weighted avg       0.93      0.84      0.87      1867

Exactitud total: 0.840


El modelo sigue siendo muy fuerte para la clase "Normal", con una alta precisión (0.98) y buen recall (0.88).

Para la clase "No normal", el modelo logra un recall de 65%, lo que significa que detecta 2 de cada 3 episodios de contaminación (CO alto).

La precisión de 0.26 indica que hay bastantes falsos positivos, es decir, el modelo a veces predice "No normal" cuando no lo es.

El F1-score de 0.38 para la clase minoritaria muestra que hay un esfuerzo efectivo pero limitado para balancear detección vs falsos positivos.

El modelo CNN-LSTM mejora notablemente el recall respecto al CNN original, pero no logra superar al LSTM balanceado, que sigue siendo el mejor modelo en balance entre detección de eventos peligrosos y estabilidad.
Este modelo aún es útil si se busca un sistema más general que combine detección local (CNN) y secuencial (LSTM), pero se queda corto en precisión al detectar CO elevado.

# Conclusión final

Sin balanceo, los modelos neuronales mostraban alta exactitud general, pero muy bajo recall en la clase minoritaria (“No normal”), lo que significa que omitían la mayoría de los episodios de contaminación.

Al aplicar class_weight para dar mayor importancia a la clase minoritaria durante el entrenamiento:

Se logró un aumento significativo del recall para “No normal” (por ejemplo, de 27% a 81% en CNN) mucha diferencia.

Se observó una reducción aceptable en la precisión general, pero con un gran beneficio en la sensibilidad del modelo ante condiciones de riesgo.

El LSTM balanceado fue el modelo con mejor desempeño general: alto recall (80%) en la clase crítica, mejor F1-score para “No normal” (0.52) y una exactitud total sólida del 91.2%. Este modelo ofrece la mejor combinación entre detección efectiva de episodios contaminantes y estabilidad en las predicciones.
